# Creating a vertical temperature profile with L'Atalante data.

This notebook contains work from the eurec4a hackathon, collaboration with Imke and Agostino. 
Contact E.Siddle@uea.ac.uk

In [ ]:
import pandas as pd
from pathlib import Path
from glob import glob
import numpy as np
from datetime import datetime,timedelta
from tqdm.notebook import tqdm
import pydap
import xarray as xr
import json

To protect the login credentials of Aeris, they need to be entered by the user into the credentials.json and called to this script using the cell below.

In [ ]:
credentials = json.loads(open('credentials.json').read()) #don't forget to add the login credentials to the credentials.json file!

username = credentials['username']
password = credentials['pass']

In [ ]:
%pylab inline
import warnings
warnings.simplefilter("ignore") 

In [ ]:
#altalante radiosonde filenames on aeris
listoffiles = ['ATL_SoundingAscentProfile_Atalante_20200203_181013.nc',# for 2020/02/03
               'ATL_SoundingAscentProfile_Atalante_20200203_172206.nc',
               'ATL_SoundingAscentProfile_Atalante_20200203_163038.nc',
               'ATL_SoundingAscentProfile_Atalante_20200203_160233.nc',
               'ATL_SoundingAscentProfile_Atalante_20200203_151307.nc',
               'ATL_SoundingAscentProfile_Atalante_20200203_142654.nc',
               'ATL_SoundingAscentProfile_Atalante_20200203_133344.nc',
               'ATL_SoundingAscentProfile_Atalante_20200203_124800.nc',
               'ATL_SoundingAscentProfile_Atalante_20200203_115848.nc',
               'ATL_SoundingAscentProfile_Atalante_20200203_111124.nc',
               'ATL_SoundingAscentProfile_Atalante_20200203_102156.nc',
               'ATL_SoundingAscentProfile_Atalante_20200203_093716.nc',
               'ATL_SoundingAscentProfile_Atalante_20200203_084812.nc',
               'ATL_SoundingAscentProfile_Atalante_20200203_080241.nc',
               'ATL_SoundingAscentProfile_Atalante_20200203_071850.nc',
               'ATL_SoundingAscentProfile_Atalante_20200203_062700.nc',
               'ATL_SoundingAscentProfile_Atalante_20200203_054035.nc',
               'ATL_SoundingAscentProfile_Atalante_20200203_043648.nc',
               'ATL_SoundingAscentProfile_Atalante_20200203_034602.nc',
               'ATL_SoundingAscentProfile_Atalante_20200203_025445.nc',
               'ATL_SoundingAscentProfile_Atalante_20200203_020400.nc',
               'ATL_SoundingAscentProfile_Atalante_20200203_012142.nc',
               'ATL_SoundingAscentProfile_Atalante_20200203_002908.nc']
listoffiles2 = ['ATL_SoundingAscentProfile_Atalante_20200203_233642.nc', #for 2020/02/02
                'ATL_SoundingAscentProfile_Atalante_20200203_224034.nc',
                'ATL_SoundingAscentProfile_Atalante_20200203_214956.nc',
                'ATL_SoundingAscentProfile_Atalante_20200203_210007.nc',
                'ATL_SoundingAscentProfile_Atalante_20200202_201504.nc',
                'ATL_SoundingAscentProfile_Atalante_20200202_193236.nc']

In [ ]:
#find max and min temperatures for later colour scale normalization
for k in range(len(listoffiles2)):
    url = 'http://' + username + ':'+ password +'@observations.ipsl.fr/thredds/dodsC/EUREC4A/SHIPS/RV-ATALANTE/RADIOSOUNDING/METEOMODEM/NetCDF/2020/2020_02_02/'+listoffiles2[k]
    store = xr.backends.PydapDataStore.open(url)
    rs = xr.open_dataset(store)
    print(rs.temperature.max(), rs.temperature.min())
    # Plot a scatterplot on a time-altitude section of temperature
    #plt.scatter(rs.flight_time.values,rs.altitude.values,c=rs.temperature.values)

In [ ]:
#find max and min temperatures for later colour scale normalization
for k in range(len(listoffiles)):
    url = 'http://' + username + ':'+ password +'@observations.ipsl.fr/thredds/dodsC/EUREC4A/SHIPS/RV-ATALANTE/RADIOSOUNDING/METEOMODEM/NetCDF/2020/2020_02_03/'+listoffiles[k]
    store = xr.backends.PydapDataStore.open(url)
    rs = xr.open_dataset(store)
    print(rs.temperature.max(), rs.temperature.min())
    # Plot a scatterplot on a time-altitude section of temperature
    #plt.scatter(rs.flight_time.values,rs.altitude.values,c=rs.temperature.values)

In [ ]:
#set path to current working directory
work_dir = Path.cwd()

In [ ]:
# open some locally stored L'atalante TSG data (previously downloaded from aeris).
tsg = xr.open_dataset(work_dir /'data'/'FNCM_TSTS_20200202.nc')
tsg2 = xr.open_dataset(work_dir /'data'/'FNCM_TSTS_20200203.nc')
tsg

In [ ]:
#identify min, max tsg temperatures
print(tsg.SSJT.min(),tsg.SSJT.max(),tsg2.SSJT.min(),tsg2.SSJT.max())

In [ ]:
#plot tsg timeseries
fig,ax = plt.subplots(1,1, figsize=(18, 15))
ax.plot(tsg.TIME, tsg.SSJT)
ax.set_ylabel('temp')
ax.set_xlabel('Time')

Plot radiosonde and TSG data for L'Atalante on 2020/02/02 and 2020/02/03.

In [ ]:
#import a few extra plotting packages
from matplotlib.collections import LineCollection
from matplotlib import colors
import matplotlib
import matplotlib.ticker as mticker

In [ ]:
#update plotting parameters for larger font
font_size=18
plt.rcParams.update({'font.size': font_size})

In [ ]:
#set colour scale normalisation and colourmap for sounding data
normalize = colors.Normalize(vmin=-35.2 , vmax=30) # scale colourbar and plots to this temp range(max and min of soundings plotted here)
cmap = cm.get_cmap('viridis')
#set up figure with subplots
fig,ax = plt.subplots(2,1, figsize=(18, 15), sharex=True, gridspec_kw={"height_ratios": [3, 1]})
ax =ax.ravel()
#loop through files list of desired soundings to plot in ax[0]
for k in range(len(listoffiles)):
    url='http://' + username + ':'+ password +'@observations.ipsl.fr/thredds/dodsC/EUREC4A/SHIPS/RV-ATALANTE/RADIOSOUNDING/METEOMODEM/NetCDF/2020/2020_02_03/'+listoffiles[k]
    store =xr.backends.PydapDataStore.open(url)
    ds=xr.open_dataset(store)
    ax[0].scatter(ds.flight_time.values,ds.altitude.values,c=ds.temperature.values, cmap = cmap, norm=normalize)
for k in range(len(listoffiles2)):
    url='http://' + username + ':'+ password +'@observations.ipsl.fr/thredds/dodsC/EUREC4A/SHIPS/RV-ATALANTE/RADIOSOUNDING/METEOMODEM/NetCDF/2020/2020_02_02/'+listoffiles2[k]
    store =xr.backends.PydapDataStore.open(url)
    ds=xr.open_dataset(store)
    ax[0].scatter(ds.flight_time.values,ds.altitude.values,c=ds.temperature.values, cmap = cmap, norm=normalize)

ax[0].set_ylim(0,10000)
#ax[1].set_ylim(-3.6,-3.4)
ax[1].plot(tsg.TIME, tsg.SSJT, color='black')
ax[1].plot(tsg2.TIME, tsg2.SSJT, color = 'black')
#ax.scatter(tsg.TIME, -(tsg.DEPH), c=tsg.SSJT, norm=normalize,cmap=cmap)
#ax.scatter(tsg2.TIME, -(tsg2.DEPH), c=tsg2.SSJT, norm=normalize,cmap=cmap)
ax[1].set_xlim(datetime.datetime(2020,2,2,18),datetime.datetime(2020,2,4,0))
ax[0].set_ylabel('Height [m]')
ax[1].set_ylabel('Sea Temperature at 3.5m depth [$^\circ$C]')
ax[1].set_xlabel('Time')
plt.xticks(rotation=45)
plt.subplots_adjust(wspace=0, hspace=0.05)

cbar = fig.colorbar( matplotlib.cm.ScalarMappable(cmap=cmap, norm=normalize),ax=ax,orientation='vertical',label='Air Temperature [$^\circ$C]')

In [ ]:
#set colour scale normalisation and colourmap for sounding data
normalizeb = colors.Normalize(vmin=20 , vmax=30) # scale colourbar and plots to this temp range(max and min of soundings plotted here)
cmap = cm.get_cmap('viridis')
#set up figure
fig,ax = plt.subplots(1,1, figsize=(18, 15))
ax.set_ylim(-5,50)
#loop through files list of desired soundings to plot in ax[0]
for k in range(len(listoffiles)):
    url='http://' + username + ':'+ password +'@observations.ipsl.fr/thredds/dodsC/EUREC4A/SHIPS/RV-ATALANTE/RADIOSOUNDING/METEOMODEM/NetCDF/2020/2020_02_03/'+listoffiles[k]
    store =xr.backends.PydapDataStore.open(url)
    ds=xr.open_dataset(store)
    ax.scatter(ds.flight_time.values,ds.altitude.values,c=ds.temperature.values, cmap = cmap, norm=normalizeb)
for k in range(len(listoffiles2)):
    url='http://' + username + ':'+ password +'@observations.ipsl.fr/thredds/dodsC/EUREC4A/SHIPS/RV-ATALANTE/RADIOSOUNDING/METEOMODEM/NetCDF/2020/2020_02_02/'+listoffiles2[k]
    store =xr.backends.PydapDataStore.open(url)
    ds=xr.open_dataset(store)
    ax.scatter(ds.flight_time.values,ds.altitude.values,c=ds.temperature.values, cmap = cmap, norm=normalizeb)

ax.scatter(tsg.TIME, -(tsg.DEPH), c=tsg.SSJT, norm=normalizeb,cmap=cmap)
ax.scatter(tsg2.TIME, -(tsg2.DEPH), c=tsg2.SSJT, norm=normalizeb,cmap=cmap)
ax.set_xlim(datetime.datetime(2020,2,2,18),datetime.datetime(2020,2,4,0))
ax.set_ylabel('Height [m]')
ax.set_xlabel('Time')
plt.xticks(rotation=45)

cbar = fig.colorbar( matplotlib.cm.ScalarMappable(cmap=cmap, norm=normalizeb),ax=ax,orientation='vertical',label='Air Temperature [$^\circ$C]')